Evan-1103204160

In [1]:
import pandas as pd
#library pandas untuk mengatur data
import matplotlib.pyplot as plt
#library matplotlib plot untuk melakukan visualisasi
import seaborn as sns
#library seaborn untuk visualisasi
from ipywidgets import interact
import ipywidgets as widgets
#ipywidgets untuk membuat interactive plot

In [2]:
#reading dataset
file_path = 'datasets/Restaurant Sales Datasets.xlsx'
df = pd.read_excel(file_path)

In [3]:
# Konversi kolom Tanggal ke format DateTime
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
# Konversi kolom Jam ke format DateTime (Hanya jam dan menit)
df['Jam'] = pd.to_datetime(df['Jam'], format='%H:%M:%S').dt.time
# Gabungkan kolom 'Tanggal' dan 'Jam' ke dalam satu kolom DateTime
df['Tanggal'] = pd.to_datetime(df['Tanggal'].astype(str) + ' ' + df['Jam'].astype(str))
# Definisikan fungsi untuk mendapatkan shift
def get_shift(time):
    if 8 <= time.hour < 14:
        return 1
    elif 14 <= time.hour < 22:
        return 2
    else:
        return 0  # Shift 0 untuk transaksi di luar jam kerja
# Terapkan fungsi shift ke DataFrame
df['Shift'] = df['Tanggal'].apply(get_shift)
# Buat DataFrame baru dengan data transaksi setiap shift
shift_data = df.groupby(['Tanggal', 'Shift']).agg({'No. Struk': 'nunique', 'Subtotal': 'sum'}).reset_index()
shift_data.columns = ['Tanggal', 'Shift', 'Jumlah Transaksi', 'Subtotal']

C:\Users\kleop\AppData\Local\Temp\ipykernel_11188\4030270761.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Tanggal'] = pd.to_datetime(df['Tanggal'])


In [4]:
# Tanggal hanya mengandung tanggal
shift_data["Tanggal"] = pd.to_datetime(shift_data["Tanggal"])
shift_data["Tanggal"] = shift_data["Tanggal"].dt.date
# Sum "Subtotal" berdasarkan "Tanggal" dan "Shift"
shift_data = shift_data.groupby(["Tanggal", "Shift"]).agg({"Subtotal": "sum", "Jumlah Transaksi": "sum"}).reset_index()
#Konversi ke datetime
shift_data["Tanggal"] = pd.to_datetime(shift_data["Tanggal"])
#menambahkan kolom hari
shift_data["Hari"] = shift_data['Tanggal'].dt.strftime('%A')
# Tanggal hanya mengandung tanggal
shift_data["Tanggal"] = shift_data["Tanggal"].dt.date

In [5]:
#mengurangi data kurang
shift_data = shift_data[shift_data['Jumlah Transaksi'] > 0]
shift_data = shift_data[shift_data['Shift'] > 0]
shift_data.describe()

,Shift,Subtotal,Jumlah Transaksi
count,1432.000000,1.432000e+03,1432.000000
mean,1.500000,5.169032e+06,28.749302
std,0.500175,2.794747e+06,13.477081
min,1.000000,3.070000e+05,2.000000
25%,1.000000,3.090750e+06,18.000000
50%,1.500000,4.599250e+06,26.000000
75%,2.000000,6.865250e+06,38.000000
max,2.000000,2.457000e+07,88.000000


In [8]:
shift_data['Tanggal'] = pd.to_datetime(shift_data['Tanggal'])
# Menggunakan Tanggal untuk mengganti kolom hari menjadi angka
shift_data['Hari'] = shift_data['Tanggal'].dt.dayofweek

In [9]:
from sklearn.model_selection import train_test_split

# Select subset of predictors
cols_to_use = ['Shift','Hari']
X = shift_data[cols_to_use]
# Select target
y = shift_data.Subtotal

# Separate shift_data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [10]:
from xgboost import XGBRegressor
my_model=XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np
from sklearn.metrics import mean_absolute_error

# Tentukan model XGBRegressor Anda dengan parameter default dan persiapkan grid parameter yang akan diuji.
model = XGBRegressor()
param_grid = {
    'n_estimators': [150, 200, 250, 300, 400, 1000]  # Anda dapat menambahkan nilai n_estimators lainnya sesuai kebutuhan
}

# Buat objek GridSearchCV.
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

# Lakukan pencarian grid pada data latih.
grid_search.fit(X_train, y_train)

# Cek hasil pencarian grid.
print("Hasil terbaik:", grid_search.best_params_)

# Gunakan nilai n_estimators terbaik ini untuk melatih model XGBRegressor Anda.
best_n_estimators = grid_search.best_params_['n_estimators']
my_model = XGBRegressor(n_estimators=best_n_estimators)
my_model.fit(X_train, y_train)

# Tampilkan nilai MAE untuk masing-masing nilai n_estimators.
mae_scores = []

for n_estimators in param_grid['n_estimators']:
    model = XGBRegressor(n_estimators=n_estimators)
    mae = -np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error'))
    mae_scores.append(mae)
    print(f"n_estimators = {n_estimators}: MAE = {mae}")

# Hasil MAE untuk setiap n_estimators
print("MAE Scores:", mae_scores)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Hasil terbaik: {'n_estimators': 150}
n_estimators = 150: MAE = 1347050.3066985437
n_estimators = 200: MAE = 1347050.3066985437
n_estimators = 250: MAE = 1347050.3066985437
n_estimators = 300: MAE = 1347050.3066985437
n_estimators = 400: MAE = 1347050.3066985437
n_estimators = 1000: MAE = 1347050.3066985437
MAE Scores: [1347050.3066985437, 1347050.3066985437, 1347050.3066985437, 1347050.3066985437, 1347050.3066985437, 1347050.3066985437]
